In [1]:
# pandas, Numpy
import pandas as pd
import numpy as np

# Import visualization packages "Matplotlib" and "Seaborn". 
import matplotlib.pyplot as plt
import seaborn as sns

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

import itertools
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Model


from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn import tree
import statsmodels.api as sm
import scipy
from sklearn.compose import ColumnTransformer, make_column_transformer
import NeuralNetwork as nn

from sklearn.model_selection import train_test_split, GridSearchCV,KFold,cross_val_score
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn import tree
import keras_tuner as kt

from sklearn.metrics import classification_report, accuracy_score, r2_score,balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
from sklearn.decomposition import PCA
import data_utils as ut

from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE,RandomOverSampler
from imblearn.under_sampling import ClusterCentroids,RandomUnderSampler

In [2]:
data=pd.read_csv('music_genre.csv')

In [4]:
data=data.dropna()

In [5]:
data=data[data['tempo']!='?']

In [6]:
data['music_genre'].value_counts()

music_genre
Rock           4561
Jazz           4521
Hip-Hop        4520
Rap            4504
Classical      4500
Anime          4497
Alternative    4495
Country        4486
Blues          4470
Electronic     4466
Name: count, dtype: int64

In [7]:
le=LabelEncoder()
data['mode']=le.fit_transform(data['mode'])


In [8]:
data['tempo']=data['tempo'].astype(float)

In [9]:
le.classes_

array(['Major', 'Minor'], dtype=object)

In [10]:
data['music_genre']=le.fit_transform(data['music_genre'])


In [11]:
y_names=le.classes_

In [12]:
data=data.drop(['obtained_date','instance_id','track_name'],axis=1).reset_index(drop=True)
data['key']=le.fit_transform(data['key'])
key_names=le.classes_
 

In [13]:
data=data[data['artist_name']!='empty_field']

In [14]:
encoder=OneHotEncoder(sparse=False)
artists_encoded=encoder.fit_transform(data['artist_name'].values.reshape(-1,1))
artist_names=encoder.categories_


In [15]:
artists_encoded=pd.DataFrame(columns=artist_names,data=artists_encoded).reset_index(drop=True)

In [16]:
artists_encoded

,!!!,"""Weird Al"" Yankovic",$NOT,$tupid Young,$uicideBoy$,(Hed) P.E.,(Sandy) Alex G,03 Greedo,04 Limited Sazabys,070 Shake,...,内田彩,夏代孝明,大原ゆい子,川田まみ,彩音,春音あいら(CV.阿澄佳奈),棗坂シキ(CV.平川大輔),楚神ウリエ(CV.近藤隆),鉤貫レム(CV.斉藤壮馬),黒崎真音
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
data=data.reset_index(drop=True)

In [18]:
data_encoded=pd.concat([artists_encoded,data],axis=1)

In [19]:
data_encoded.select_dtypes(include=['object']).columns

Index(['artist_name'], dtype='object')

In [20]:
y=data_encoded['music_genre']
y=pd.get_dummies(y).astype(int)
y=y.reset_index(drop=True)
y

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
42786,0,0,0,0,0,0,1,0,0,0
42787,0,0,0,0,0,0,1,0,0,0
42788,0,0,0,0,0,0,1,0,0,0
42789,0,0,0,0,0,0,1,0,0,0


In [21]:
X=data_encoded.drop(['music_genre','artist_name'],axis=1).reset_index(drop=True)

In [22]:
X.columns=X.columns.astype(str)

In [23]:
# Set the figure size 
# data_heat=data.copy()

# plt.figure(figsize=(15, 10))

# # Correlation matrix / heatmap
# sns.heatmap(data_heat.corr(),cmap='Blues', annot=True,fmt=".2f")
# # Display the heatmap
# plt.show()

In [24]:
# data.plot.scatter(x='instrumentalness',y='danceability', c='music_genre', colormap='winter')

In [25]:
# data.columns

In [26]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=3)

In [27]:
scaler=StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [28]:
X_test_scaled=scaler.transform(X_test)
X_train=scaler.transform(X_train)

In [29]:
len(X_train)

32093

In [30]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=20,
        max_value=60,
        step=5), activation=activation, input_dim=len(X.columns)))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=20,
            max_value=60,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=10, activation="softmax"))

    # Compile the model
    nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

    return nn_model

In [31]:

tuner = kt.Hyperband(
        create_model,
        objective='loss',
        max_epochs=20,
        overwrite=True,
        hyperband_iterations=2)
        
stop_early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)


In [32]:
tuner.search(X_train,y_train,epochs=20,batch_size=128,validation_data=(X_test_scaled,y_test), callbacks=[stop_early])

Trial 60 Complete [00h 00m 12s]
loss: 0.2534867823123932

Best loss So Far: 0.1931888610124588
Total elapsed time: 00h 06m 27s


In [33]:
top_hyper = tuner.get_best_hyperparameters(3)

for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'first_units': 55, 'num_layers': 1, 'units_0': 60, 'units_1': 20, 'units_2': 30, 'units_3': 35, 'units_4': 55, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0042'}
{'activation': 'tanh', 'first_units': 35, 'num_layers': 1, 'units_0': 45, 'units_1': 35, 'units_2': 60, 'units_3': 35, 'units_4': 50, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0043'}
{'activation': 'tanh', 'first_units': 45, 'num_layers': 1, 'units_0': 60, 'units_1': 45, 'units_2': 60, 'units_3': 45, 'units_4': 20, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0018'}


In [34]:
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

335/335 - 0s - 1ms/step - accuracy: 0.7740 - loss: 0.6552
Loss: 0.6552392244338989, Accuracy: 0.7739764451980591
335/335 - 0s - 1ms/step - accuracy: 0.7735 - loss: 0.6450
Loss: 0.6450018882751465, Accuracy: 0.7735090851783752
335/335 - 0s - 1ms/step - accuracy: 0.7754 - loss: 0.6731
Loss: 0.6730644106864929, Accuracy: 0.7753785848617554


In [36]:
model=tf.keras.models.Sequential()
input_nodes=len(X.columns)
model.add(tf.keras.layers.Dense(units=55,activation='tanh',input_dim=input_nodes))

model.add(tf.keras.layers.Dense(units=60,activation='tanh'))
# model.add(tf.keras.layers.Dense(units=50,activation='tanh'))
# model.add(tf.keras.layers.Dense(units=45,activation='tanh'))
# model.add(tf.keras.layers.Dense(units=45,activation='relu'))

model.add(tf.keras.layers.Dense(units=10,activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 55)             │       363,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 60)             │         3,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 10)             │           610 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 367,850 (1.40 MB)

 Trainable params: 367,850 (1.40 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
history=model.fit(X_train,y_train, epochs=200,batch_size=128,validation_data=(X_test_scaled,y_test))

Epoch 1/200
251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4795 - loss: 1.5739 - val_accuracy: 0.7573 - val_loss: 0.6039
Epoch 2/200
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8243 - loss: 0.4053 - val_accuracy: 0.7687 - val_loss: 0.5548
Epoch 3/200
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8473 - loss: 0.3062 - val_accuracy: 0.7805 - val_loss: 0.5475
Epoch 4/200
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8578 - loss: 0.2776 - val_accuracy: 0.7805 - val_loss: 0.5478
Epoch 5/200
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8602 - loss: 0.2642 - val_accuracy: 0.7770 - val_loss: 0.5424
Epoch 6/200
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8601 - loss: 0.2604 - val_accuracy: 0.7787 - val_loss: 0.5555
Epoch 7/200
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8661 - loss: 0.2492 - val_accuracy: 0.7831 - val_loss: 0.5534
Epoch 8/200
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8693 - loss: 0.2435 - val_accu

In [38]:
modelloss,modelaccuracy=model.evaluate(X_test_scaled,y_test,verbose=2)
print(f'model loss: {modelloss},  model accuracy: {modelaccuracy}')

335/335 - 0s - 568us/step - accuracy: 0.7320 - loss: 1.6881
model loss: 1.6880955696105957,  model accuracy: 0.7320059537887573


In [191]:
xtest=X[30000:30001]
xtest

,x0_!!!,"x0_""Weird Al"" Yankovic",x0_$NOT,x0_$tupid Young,x0_$uicideBoy$,x0_(Hed) P.E.,x0_(Sandy) Alex G,x0_03 Greedo,x0_04 Limited Sazabys,x0_070 Shake,...,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
30000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,250760.0,0.618,0.694,3,0.124,-13.482,0,0.043,104.983,0.464


In [192]:
test=scaler.transform(xtest)
test

array([[-0.01934045,  0.        , -0.00966887, ..., -0.50149464,
        -0.49003975,  0.02894297]])

In [190]:
data.iloc[30000]

artist_name         The National
popularity                  54.0
acousticness               0.326
danceability               0.556
duration_ms             250760.0
energy                     0.618
instrumentalness           0.694
key                            3
liveness                   0.124
loudness                 -13.482
mode                           0
speechiness                0.043
tempo                    104.983
valence                    0.464
music_genre                    9
Name: 30000, dtype: object

In [193]:
prediction=model.predict(test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [194]:
prediction

array([[8.7309934e-12, 7.2190700e-12, 3.4397370e-12, 2.2072653e-10,
        6.3390139e-08, 5.1057960e-08, 2.4796507e-14, 1.3963729e-10,
        1.1102490e-10, 9.9999988e-01]], dtype=float32)

In [195]:
predictions_df = pd.DataFrame(columns=y_names, data=prediction)
predictions_df

,Alternative,Anime,Blues,Classical,Country,Electronic,Hip-Hop,Jazz,Rap,Rock
0,8.730993e-12,7.219070e-12,3.439737e-12,2.207265e-10,6.339014e-08,5.105796e-08,2.479651e-14,1.396373e-10,1.110249e-10,1.0
